In [11]:
pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-ptkide2f
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-ptkide2f
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [14]:
%reload_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [16]:
%%cu
#include <iostream>
using namespace std;

__global__ void vectorAdd(int *a, int *b, int *result, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < n) {
        result[tid] = a[tid] + b[tid];
    }
}

int main() {
    int *a, *b, *c;
    int *a_dev, *b_dev, *c_dev;
    int n = 1 << 24; // Example large number

    a = new int[n];
    b = new int[n];
    c = new int[n];
    int size = n * sizeof(int);

    cudaMalloc(&a_dev, size);
    cudaMalloc(&b_dev, size);
    cudaMalloc(&c_dev, size);

    // Initialize arrays with values
    for (int i = 0; i < n; i++) {
        a[i] = 1;
        b[i] = 2;
    }

    // Copy data from host to device
    cudaMemcpy(a_dev, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, b, size, cudaMemcpyHostToDevice);

    int threads = 1024;
    int blocks = (n + threads - 1) / threads;

    // Execute kernel on GPU
    vectorAdd<<<blocks, threads>>>(a_dev, b_dev, c_dev, n);

    // Copy result back to host
    cudaMemcpy(c, c_dev, size, cudaMemcpyDeviceToHost);

    // Verify the result
    bool success = true;
    for (int i = 0; i < n; i++) {
        if (c[i] != a[i] + b[i]) {
            success = false;
            std::cout << "Error at position " << i << std::endl;
            break;
        }
    }

    if (success) {
        std::cout << "Vector addition successful!" << std::endl;
    }

    // Free memory
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    delete[] a;
    delete[] b;
    delete[] c;

    return 0;
}

Error at position 0

